In [6]:
import numpy as np
import re

In [7]:
import pickle
def save_obj(obj, name ):
    with open('data/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('data/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [8]:
data = load_obj('posts_training_data')

In [9]:
features = np.array([list(feature.values()) for feature in data['feature']])
target = data['sentiment']
print(features.shape)

(338, 11)


In [10]:
for key in data['feature'][0].keys():
    print(key)

num_slang
question_mark
neg
pos
number_number
pos_strength
neg_strength
pos_vs_neg_strength
pos_vs_neg
exclamation
avg_word_len


In [43]:
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import csv
import os.path
from sklearn import preprocessing
def grid_search(clf, features_dict_list, target, param_grid, cv=None, 
                scoring=None, feature_scaled=False, output=True, output_file=None, minmax=False):
    if not cv:
        cv = 10
    if not scoring:
        scoring = 'accuracy'
    if not output_file:
        output_file = 'data/classifier_selection.csv'
    features = np.array([list(feature.values()) for feature in features_dict_list])
    if feature_scaled:
        features = preprocessing.scale(features)
    if minmax:
        scaler = MinMaxScaler()
        features = scaler.fit_transform(features)
    feature_names = ', '.join(features_dict_list[0].keys())
    gs_clf = GridSearchCV(estimator=clf, param_grid=param_grid,
                   n_jobs=-1, cv=cv,scoring=scoring)
    gs_clf.fit(features, target)
    grid_scores = gs_clf.grid_scores_
    # write training data to .csv
    file_exist = os.path.isfile(output_file) 
    if not file_exist and output:
        with open(output_file, 'w') as csvfile:
            writer = csv.writer(csvfile)

            writer.writerow(["classifier", "parameters", "features", "feature_scaled", 
                             "cv", "avg_accuracy", "std_accuracy"])
            for score in grid_scores: 
                params = score[0]
                avg_accuracy = score[1]
                std_accuracy = np.std(score[2])
                writer.writerow([type(clf).__name__, params, feature_names,
                                 feature_scaled, cv, avg_accuracy, std_accuracy])
    elif file_exist and output:
        with open(output_file, 'a') as csvfile:
            writer = csv.writer(csvfile)
            for score in grid_scores: 
                params = score[0]
                avg_accuracy = score[1]
                std_accuracy = np.std(score[2])
                writer.writerow([type(clf).__name__, params, feature_names,
                                 feature_scaled, cv, avg_accuracy, std_accuracy])
    else:
        pass
    print(gs_clf.best_score_)
    print(gs_clf.best_params_)
    return gs_clf

### Get subsets of feature dictionary

In [12]:
from itertools import chain, combinations
def all_subsets(ss):
  return(chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1))))

In [15]:
for subset in all_subsets(data['feature'][0].keys()):
  print(list(subset))

[]
['num_slang']
['question_mark']
['neg']
['pos']
['number_number']
['pos_strength']
['neg_strength']
['pos_vs_neg_strength']
['pos_vs_neg']
['exclamation']
['avg_word_len']
['num_slang', 'question_mark']
['num_slang', 'neg']
['num_slang', 'pos']
['num_slang', 'number_number']
['num_slang', 'pos_strength']
['num_slang', 'neg_strength']
['num_slang', 'pos_vs_neg_strength']
['num_slang', 'pos_vs_neg']
['num_slang', 'exclamation']
['num_slang', 'avg_word_len']
['question_mark', 'neg']
['question_mark', 'pos']
['question_mark', 'number_number']
['question_mark', 'pos_strength']
['question_mark', 'neg_strength']
['question_mark', 'pos_vs_neg_strength']
['question_mark', 'pos_vs_neg']
['question_mark', 'exclamation']
['question_mark', 'avg_word_len']
['neg', 'pos']
['neg', 'number_number']
['neg', 'pos_strength']
['neg', 'neg_strength']
['neg', 'pos_vs_neg_strength']
['neg', 'pos_vs_neg']
['neg', 'exclamation']
['neg', 'avg_word_len']
['pos', 'number_number']
['pos', 'pos_strength']
['pos',

In [16]:
subsets_tuple = all_subsets(data['feature'][0].keys())
subsets = [subset for idx, subset in enumerate(subsets_tuple)]
subsets.pop(0)

()

In [17]:
subsets

[('num_slang',),
 ('question_mark',),
 ('neg',),
 ('pos',),
 ('number_number',),
 ('pos_strength',),
 ('neg_strength',),
 ('pos_vs_neg_strength',),
 ('pos_vs_neg',),
 ('exclamation',),
 ('avg_word_len',),
 ('num_slang', 'question_mark'),
 ('num_slang', 'neg'),
 ('num_slang', 'pos'),
 ('num_slang', 'number_number'),
 ('num_slang', 'pos_strength'),
 ('num_slang', 'neg_strength'),
 ('num_slang', 'pos_vs_neg_strength'),
 ('num_slang', 'pos_vs_neg'),
 ('num_slang', 'exclamation'),
 ('num_slang', 'avg_word_len'),
 ('question_mark', 'neg'),
 ('question_mark', 'pos'),
 ('question_mark', 'number_number'),
 ('question_mark', 'pos_strength'),
 ('question_mark', 'neg_strength'),
 ('question_mark', 'pos_vs_neg_strength'),
 ('question_mark', 'pos_vs_neg'),
 ('question_mark', 'exclamation'),
 ('question_mark', 'avg_word_len'),
 ('neg', 'pos'),
 ('neg', 'number_number'),
 ('neg', 'pos_strength'),
 ('neg', 'neg_strength'),
 ('neg', 'pos_vs_neg_strength'),
 ('neg', 'pos_vs_neg'),
 ('neg', 'exclamation')

### SVM

In [62]:
from sklearn import svm
Cs = np.logspace(-1, 2, 10)
gammas = [0.001, 0.0001]
svc = svm.SVC(probability=True)
#clf = grid_search(svc,data['feature'],data['sentiment'],dict(C=Cs))

In [63]:
clf = grid_search(svc,data['feature'],data['sentiment'],dict(C=Cs), cv=5, feature_scaled=True,output=False)

0.772189349112


In [58]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(svc,subset_feature,data['sentiment'],dict(C=Cs))
    grid_search(svc,subset_feature,data['sentiment'],dict(C=Cs),cv=5)
    grid_search(svc,subset_feature,data['sentiment'],dict(C=Cs),feature_scaled=True)
    grid_search(svc,subset_feature,data['sentiment'],dict(C=Cs),cv=5,feature_scaled=True)

('exclamation',)
0.621301775148
0.621301775148


/Users/Johnson/Documents/nit/thesis/crawler/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


0.621301775148


/Users/Johnson/Documents/nit/thesis/crawler/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


0.621301775148
('question_mark',)
0.674556213018
0.674556213018


...

0.757396449704
('question_mark', 'pos_strength', 'avg_word_len', 'neg', 'neg_strength')
0.736686390533
0.704142011834
0.789940828402
0.789940828402
('question_mark', 'pos_strength', 'avg_word_len', 'neg', 'number_number')
0.62426035503
0.621301775148
0.742603550296
0.745562130178
('question_mark', 'pos_strength', 'avg_word_len', 'neg_strength', 'number_number')
0.668639053254
0.659763313609
0.757396449704
0.766272189349
('question_mark', 'pos_strength', 'neg', 'neg_strength', 'number_number')
0.671597633136
0.662721893491
0.760355029586
0.763313609467
('question_mark', 'pos_vs_neg_strength', 'pos', 'num_slang', 'avg_word_len')
0.742603550296
0.736686390533
0.730769230769
0.742603550296
('question_mark', 'pos_vs_neg_strength', 'pos', 'num_slang', 'neg')
0.742603550296
0.739644970414
0.769230769231
0.781065088757
('question_mark', 'pos_vs_neg_strength', 'pos', 'num_slang', 'neg_strength')
0.715976331361
0.710059171598
0.760355029586
0.772189349112
('question_mark', 'pos_vs_neg_strength',

/Users/Johnson/Documents/nit/thesis/crawler/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


0.757396449704


/Users/Johnson/Documents/nit/thesis/crawler/lib/python3.5/site-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


0.760355029586
('exclamation', 'question_mark', 'pos_strength', 'avg_word_len', 'neg', 'neg_strength')
0.727810650888
0.715976331361
0.786982248521
0.792899408284
('exclamation', 'question_mark', 'pos_strength', 'avg_word_len', 'neg', 'number_number')
0.639053254438
0.621301775148
0.745562130178
0.754437869822
('exclamation', 'question_mark', 'pos_strength', 'avg_word_len', 'neg_strength', 'number_number')
0.677514792899
0.677514792899
0.757396449704
0.778106508876
('exclamation', 'question_mark', 'pos_strength', 'neg', 'neg_strength', 'number_number')
0.683431952663
0.677514792899
0.751479289941
0.763313609467
('exclamation', 'question_mark', 'pos_vs_neg_strength', 'pos', 'num_slang', 'avg_word_len')
0.730769230769
0.724852071006
0.733727810651
0.742603550296
('exclamation', 'question_mark', 'pos_vs_neg_strength', 'pos', 'num_slang', 'neg')
0.733727810651
0.727810650888
0.769230769231
0.778106508876
('exclamation', 'question_mark', 'pos_vs_neg_strength', 'pos', 'num_slang', 'neg_stren

In [70]:
clf.predict_proba(list(data['feature'][0].values()))

/Users/Johnson/Documents/nit/thesis/crawler/lib/python3.5/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.26056572,  0.73943428]])

### AdaBoost

In [77]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

base_estimators = [DecisionTreeClassifier(max_depth=1),
                   DecisionTreeClassifier(max_depth=2),
                   DecisionTreeClassifier(max_depth=3)]
n_estimators = np.linspace(1,20,10).astype(int)
# Create and fit an AdaBoosted decision tree
bdt = AdaBoostClassifier(learning_rate = 0.1)

In [ ]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(bdt,subset_feature,data['sentiment'], output_file = 'data/adaboost_classifier_selection.csv',
                param_grid=dict(base_estimator=base_estimators,n_estimators=n_estimators))
    grid_search(bdt,subset_feature,data['sentiment'], output_file = 'data/adaboost_classifier_selection.csv',
                param_grid=dict(base_estimator=base_estimators,n_estimators=n_estimators),feature_scaled=True)

### Logistic Regression

In [18]:
from sklearn import linear_model
Cs = np.logspace(-1, 2, 20)
lr = linear_model.LogisticRegression()

In [26]:
grid_search(lr,data['feature'],data['sentiment'], dict(C=Cs),output=False)

0.778106508876
{'C': 0.10000000000000001}


GridSearchCV(cv=10, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([   0.1    ,    0.14384,    0.20691,    0.29764,    0.42813,
          0.61585,    0.88587,    1.27427,    1.83298,    2.63665,
          3.79269,    5.45559,    7.8476 ,   11.28838,   16.23777,
         23.35721,   33.59818,   48.3293 ,   69.51928,  100.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [ ]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(lr,subset_feature,data['sentiment'], dict(C=Cs), output_file = 'data/logistic_classifier_selection.csv')
    grid_search(lr,subset_feature,data['sentiment'], dict(C=Cs),
                output_file = 'data/logistic_classifier_selection.csv', feature_scaled=True)

### Naive Bayes

In [29]:
from sklearn.naive_bayes import GaussianNB

GNB = GaussianNB()

In [30]:
grid_search(GNB,data['feature'],data['sentiment'], {},output=False)

0.689349112426
{}


GridSearchCV(cv=10, error_score='raise', estimator=GaussianNB(),
       fit_params={}, iid=True, n_jobs=-1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [ ]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(GNB,subset_feature,data['sentiment'], {}, output_file = 'data/naive_bayes_classifier_selection.csv')
    grid_search(GNB,subset_feature,data['sentiment'], {},
                output_file = 'data/naive_bayes_classifier_selection.csv', feature_scaled=True)

In [55]:
from sklearn.naive_bayes import MultinomialNB
alphas = np.logspace(-3, 1, 20)
MNB = MultinomialNB()

In [56]:
grid_search(MNB,data['feature'],data['sentiment'], dict(alpha=alphas),minmax=True,feature_scaled=True,output=False)

0.647928994083
{'alpha': 0.001}


GridSearchCV(cv=10, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'alpha': array([  1.00000e-03,   1.83298e-03,   3.35982e-03,   6.15848e-03,
         1.12884e-02,   2.06914e-02,   3.79269e-02,   6.95193e-02,
         1.27427e-01,   2.33572e-01,   4.28133e-01,   7.84760e-01,
         1.43845e+00,   2.63665e+00,   4.83293e+00,   8.85867e+00,
         1.62378e+01,   2.97635e+01,   5.45559e+01,   1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [ ]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(MNB,subset_feature,data['sentiment'], dict(alpha=alphas), minmax=True,
                output_file = 'data/naive_bayes_classifier_selection.csv')
    grid_search(MNB,subset_feature,data['sentiment'], dict(alpha=alphas),
                output_file = 'data/naive_bayes_classifier_selection.csv', minmax=True,feature_scaled=True)

### Decision Tree

In [66]:
from sklearn import tree
max_depths = np.arange(1,len(data['feature'][0]))
dt = tree.DecisionTreeClassifier()

In [67]:
grid_search(dt,data['feature'],data['sentiment'], dict(max_depth=max_depths),output=False)

0.751479289941
{'max_depth': 3}


GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [68]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(dt,subset_feature,data['sentiment'], dict(max_depth=max_depths), output=False)

('num_slang',)
0.621301775148
{'max_depth': 5}
('question_mark',)
0.677514792899
{'max_depth': 1}
('neg',)
0.689349112426
{'max_depth': 2}
('pos',)
0.621301775148
{'max_depth': 1}
('number_number',)
0.618343195266
{'max_depth': 2}
('pos_strength',)
0.630177514793
{'max_depth': 4}
('neg_strength',)
0.653846153846
{'max_depth': 1}
('pos_vs_neg_strength',)
0.742603550296
{'max_depth': 2}
('pos_vs_neg',)
0.721893491124
{'max_depth': 1}
('exclamation',)
0.621301775148
{'max_depth': 1}
('avg_word_len',)
0.621301775148
{'max_depth': 1}
('num_slang', 'question_mark')
0.692307692308
{'max_depth': 3}
('num_slang', 'neg')
0.689349112426
{'max_depth': 2}
('num_slang', 'pos')
0.621301775148
{'max_depth': 1}
('num_slang', 'number_number')
0.615384615385
{'max_depth': 1}
('num_slang', 'pos_strength')
0.633136094675
{'max_depth': 5}
('num_slang', 'neg_strength')
0.653846153846
{'max_depth': 1}
('num_slang', 'pos_vs_neg_strength')
0.742603550296
{'max_depth': 2}
('num_slang', 'pos_vs_neg')
0.7278106508

Process ForkPoolWorker-1116:
Process ForkPoolWorker-1113:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/loc

KeyboardInterrupt: 

In [ ]:
for subset in subsets:
    subset_feature = []
    for feature in data['feature']:
        subset_feature.append({k:feature[k] for k in subset})
    subset_feature = np.array(subset_feature)
    print(subset)
    grid_search(dt,subset_feature,data['sentiment'], {}, output_file = 'data/decision_tree_classifier_selection.csv')
    grid_search(dt,subset_feature,data['sentiment'], {},
                output_file = 'data/decision_tree_classifier_selection.csv', feature_scaled=True)